## This notebook does slew simulations, and check all aos components (M1M3, M2, hexapods) behavior during the slew-and-track process

This is expected to work both for SUMMIT and NCSA

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lsst.ts import salobj
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

from lsst.ts.idl.enums import MTHexapod

from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

from aosTools import *

plt.jet();

<Figure size 432x288 with 0 Axes>

In [3]:
#summit = 1 #use this for summit testing
summit = 0 #use this for NCSA

In [4]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ["LSST_DDS_DOMAIN_ID"])

file:///home/bxin/ts_ddsconfig/config/ospl-shmem.xml
ncsa
0


In [5]:
from lsst_efd_client import EfdClient

if summit:
    client = EfdClient('summit_efd')
else:
    client = EfdClient('ncsa_teststand_efd')
csc_index = 1

In [6]:
#index is an integter which helps avoid multple users starting same controller
script = salobj.Controller("Script", index=7)

In [7]:
mtcs = MTCS(script.domain)
mtcs.set_rem_loglevel(40)

In [8]:
await mtcs.start_task

electrical DDS read queue is filling: 73 of 100 elements
electrical DDS read queue is full (100 elements); data may be lost
elevation DDS read queue is full (100 elements); data may be lost
application DDS read queue is filling: 75 of 100 elements
application DDS read queue is full (100 elements); data may be lost
cameraCableWrap DDS read queue is full (100 elements); data may be lost
actuators DDS read queue is filling: 76 of 100 elements
actuators DDS read queue is full (100 elements); data may be lost
azimuth DDS read queue is full (100 elements); data may be lost
timeAndDate DDS read queue is full (100 elements); data may be lost
powerSupplyData DDS read queue is full (100 elements); data may be lost


[None, None, None, None, None, None, None, None, None, None]

mountStatus DDS read queue is full (100 elements); data may be lost
pidData DDS read queue is full (100 elements); data may be lost
mountPosition DDS read queue is full (100 elements); data may be lost
inclinometerData DDS read queue is full (100 elements); data may be lost


In [9]:
mtcs.components_attr

imsData DDS read queue is full (100 elements); data may be lost
currentTargetStatus DDS read queue is full (100 elements); data may be lost
hardpointMonitorData DDS read queue is full (100 elements); data may be lost
hardpointActuatorData DDS read queue is full (100 elements); data may be lost
gyroData DDS read queue is full (100 elements); data may be lost
forceActuatorData DDS read queue is full (100 elements); data may be lost
accelerometerData DDS read queue is full (100 elements); data may be lost


['mtmount',
 'mtptg',
 'mtaos',
 'mtm1m3',
 'mtm2',
 'mthexapod_1',
 'mthexapod_2',
 'mtrotator',
 'mtdome',
 'mtdometrajectory']

In [10]:
ptg = mtcs.rem.mtptg
mount = mtcs.rem.mtmount
rot = mtcs.rem.mtrotator
camhex = mtcs.rem.mthexapod_1
m2hex = mtcs.rem.mthexapod_2
m1m3 = mtcs.rem.mtm1m3
m2 = mtcs.rem.mtm2
#aos = mtcs.rem.mtaos

### Check that all components are ready

In [12]:
await checkAOSCompStates(m1m3, m2, camhex, m2hex)

starting with: M1M3 state State.ENABLED 2021-05-18 16:56:45.545589248
          detailed state DetailedState.ACTIVE 2021-05-18 16:56:45.545640192
staring with: m2 state State.ENABLED 2021-05-06 17:22:57.171261184
staring with: cam hex state State.ENABLED 2021-05-18 17:34:17.355547392
          cam hex state EnabledSubstate.STATIONARY 2021-05-18 17:34:17.355837952
staring with: M2 hex state State.ENABLED 2021-05-18 17:34:17.709411072
          hex state EnabledSubstate.STATIONARY 2021-05-18 17:34:17.709662720


In [13]:
await checkSlewCompStates(ptg, mount, rot)

staring with: ptg state State.ENABLED 2021-05-07 15:26:01.249896960
staring with: mount state State.ENABLED 2021-05-13 16:52:45.512967168
staring with: rot state State.ENABLED 2021-05-13 16:52:58.822613760


### Get Ptg ready

In [121]:
await salobj.set_summary_state(ptg, salobj.State.ENABLED)

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

### Get Mount ready

In [85]:
await salobj.set_summary_state(mount, salobj.State.STANDBY)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [122]:
await salobj.set_summary_state(mount, salobj.State.ENABLED)

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

### Get Rotator ready

In [123]:
a = await mount.tel_elevation.next(flush=True, timeout=5)
print("mount elevation Angle = ", a.actualPosition)
a = await mount.tel_azimuth.next(flush=True, timeout=5)
print("mount azimuth angle = ", a.actualPosition)
a = await mount.tel_cameraCableWrap.next(flush=True, timeout=5)
print("CCW angle = ", a.actualPosition, " Needs to be within 2.2 deg of rotator angle ")
b = await rot.tel_rotation.next(flush=True, timeout=5)
print("rot angle = ", b.actualPosition, "   diff = ", (b.actualPosition - a.actualPosition))

mount elevation Angle =  80.0
mount azimuth angle =  0.0
CCW angle =  -7.318164786410081  Needs to be within 2.2 deg of rotator angle 
rot angle =  -27.44068801303646    diff =  -20.122523226626377


In [72]:
a = await rot.evt_errorCode.aget()
print(a.errorReport)

Camera cable wrap not following closely enough: error # 7 = -5.382058246874115 > 2.2 deg


In [71]:
a = await mount.evt_cameraCableWrapState.aget(timeout=5.)
print(a.state)
a = await mount.evt_cameraCableWrapFollowing.aget(timeout=5.)
print(a.enabled)

TimeoutError: 

In [55]:
await mount.cmd_enableCameraCableWrapFollowing.set_start()

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=690344821, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result="Failed: RuntimeError('<DeviceId.CAMERA_CABLE_WRAP: 4> off')")

In [124]:
await rot.cmd_clearError.set_start()

In [125]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

### Get M1M3 ready (Mount telemetry mode)

In [92]:
#Move to zenith (so that we can start m1m3 with LUT in mount telemetry mode)
await mount.cmd_moveToTarget.set_start(azimuth=0, elevation=90)

In [16]:
await m1m3.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
print('Re-enabling M1M3')
#await salobj.set_summary_state(m1m3, salobj.State.STANDBY) 
await salobj.set_summary_state(m1m3, salobj.State.ENABLED, settingsToApply = 'Default') #enable m1m3

In [14]:
await readyM1M3(m1m3)

m1m3 inclinometer Angle =  90.0
M1M3 is raised
m1m3 z position =  0.0
Magnitude of the m1m3 force balance system 3479.10546875


### Get M2 ready

In [17]:
await m2.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await salobj.set_summary_state(m2, salobj.State.ENABLED) 

In [15]:
await readyM2(m2)

m2 inclinometer angle =  0.033274132462053685
Status of the M2 force balance system True
clear any M2 activeopticForces (or any other hunman-applied forces)


### Get camHex ready

In [26]:
target_evt = await mount.evt_target.aget(timeout=5.)
print("Mount target elevation = ", target_evt.elevation, "  @  ", pd.to_datetime(target_evt.private_sndStamp, unit='s'))
print("Mount target azimuth   = ", target_evt.azimuth, "  @  ", pd.to_datetime(target_evt.private_sndStamp, unit='s'))
target_evt = await rot.evt_target.aget(timeout=5.)
print("Rotator target position = ", target_evt.position, "  @  ", pd.to_datetime(target_evt.private_sndStamp, unit='s'))

Mount target elevation =  90.0   @   2021-05-25 22:18:23.944214272
Mount target azimuth   =  0.0   @   2021-05-25 22:18:23.944214272
Rotator target position =  21.329276563855764   @   2021-05-07 12:51:27.033421312


In [ ]:
#if any of the above doesn't work
await mount.cmd_moveToTarget.set_start(azimuth=0, elevation=90)
await rot.cmd_move.set_start(position=0)

In [27]:
await camhex.evt_heartbeat.next(flush=True, timeout=5)

In [29]:
await readyHexaForAOS(camhex)

settingsVersion =  default.yaml:v0.7.2-0-g4375e27 2021-05-18 17:34:17.151204864
pivot at (0, 0, 500000) microns 
maxXY =  11400.0 microns, maxZ=  13100.0  microns
maxUV =  0.36 deg, maxW=  0.1  deg
compsensation mode enabled? True 2021-05-18 17:36:34.992618752
hexa in position? True 2021-05-25 22:52:24.580910592
Current Hexapod position
     -0.94    -652.94     395.57  -0.017752   0.000000  -0.000000
hexa in position? False 2021-05-25 22:52:24.780633088
hexa in position? True 2021-05-25 22:52:24.881603072
Current Hexapod position
     -0.88    -652.98     295.58  -0.017751  -0.000000   0.000000
Uncompensated position
      0.00       0.00       0.00      0.000000   0.000000   0.000000    2021-05-25 22:52:24.691219456
Compensated position
     -0.93    -652.98     295.56      -0.017752   0.000000   0.000000    2021-05-25 22:52:24.691528448
Does the hexapod has enough inputs to do LUT compensation? (If the below times out, we do not.)
mount elevation =  90.0
mount azimth =  0.0
rotator 

### Get m2Hex ready

In [31]:
await readyHexaForAOS(m2hex)

settingsVersion =  default.yaml:v0.7.2-0-g4375e27 2021-05-18 17:34:17.503598336
pivot at (0, 0, 500000) microns 
maxXY =  10500.0 microns, maxZ=  8900.0  microns
maxUV =  0.175 deg, maxW=  0.05  deg
compsensation mode enabled? True 2021-05-18 17:38:00.095350528
hexa in position? True 2021-05-25 22:52:52.921856768
Current Hexapod position
     -1.84    -572.58     404.65  -0.006649   0.000000   0.000000
hexa in position? False 2021-05-25 22:52:53.124936704
hexa in position? True 2021-05-25 22:52:53.226160640
Current Hexapod position
     -1.87    -572.60     304.68  -0.006649  -0.000000   0.000000
Uncompensated position
      0.00       0.00       0.00      0.000000   0.000000   0.000000    2021-05-25 22:52:53.029085952
Compensated position
     -1.84    -572.60     304.70      -0.006649   0.000000   0.000000    2021-05-25 22:52:53.029367552
Does the hexapod has enough inputs to do LUT compensation? (If the below times out, we do not.)
mount elevation =  90.0
mount azimth =  0.0
rotator

In [32]:
end = Time(datetime.now(), scale='tai')
start = end - timedelta(seconds=1000)
df = await client.select_time_series('lsst.sal.ESS.temperature8Ch', '*', start, end, csc_index)
if len(df)>0:
    fig, ax = plt.subplots(1,1, figsize=(15,4))
    for i in range(8):
        plt.plot(getattr(df, 'temperatureC%02d'%i))
    plt.grid()
else:
    print('No temperature data on the camera hexapod in the past 1000s.')

No temperature data on the camera hexapod in the past 1000s.


### Do a few slews

In [33]:
#comment and uncomment the below depending on which components are expected to be on
#by default, MTCS checks on every component (during the slew).
mtcs.check.mtaos = False
mtcs.check.mtm1m3 = True
mtcs.check.mtm2 = True
mtcs.check.mthexapod_1 = True
mtcs.check.mthexapod_2 = True
mtcs.check.mtdome = False
mtcs.check.mtdometrajectory = False

#### Do 4 slews, then stop tracking

In [101]:
if not summit: #the large slews here will cause M1M3 fault on the summit
    dec = -34.
    for j in range(2):
        time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.next(flush=True, timeout=5)
        ra = time_and_date.lst + 0.5
        for i in range(2):
            print(ra, dec)
            await mtcs.slew_icrs(ra=ra, dec=dec, rot=0., rot_type=RotType.PhysicalSky)
            await asyncio.sleep(39.)
            ra -= 3.5/15.0

    await ptg.cmd_stopTracking.set_start(timeout=5.)

15.760852914598091 -34.0
15.527519581264759 -34.0
15.793479625278861 -34.0
15.560146291945529 -34.0


NameError: name 'usePtg' is not defined

In [118]:
dec = -34.
for j in range(2):
    time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.next(flush=True, timeout=5)
    ra = time_and_date.lst + 0.5
    for i in range(2):
        
        aa = await mount.tel_elevation.next(flush=True, timeout=5.)
        current_el = aa.actualPosition
        
        obs_time = salobj.astropy_time_from_tai_unix(salobj.current_tai() + 0.) #with 0s delay
        azel = mtcs.azel_from_radec(ra=ra, dec=dec, time=obs_time)
        target_el = azel.alt.value
        
        print('moving from elevation %.1f deg to %.1f deg'%(current_el, target_el), Time.now())
        await moveMountConstantV(mount, current_el, target_el)
        print('start a slew', Time.now())

        b = await mount.tel_cameraCableWrap.next(flush=True, timeout=5) #CCW doesn't follow rotator!!!
        await mtcs.slew_icrs(ra=ra, dec=dec, rot=b.actualPosition, rot_type=RotType.PhysicalSky)
        await asyncio.sleep(39.)
        await ptg.cmd_stopTracking.set_start(timeout=5.)
        ra -= 3.5/15.0
    break

moving from elevation 80.9 deg to 82.3 deg
This will run for 25 seconds
81.0229462346934 0.0 0.016689006487528482
81.14900240913062 0.0 0.041900241374969484
81.2812037429713 0.0 0.06834050814310709
81.41393345211971 0.0 0.09488644997278849
81.5468292794131 0.0 0.1214656154314677
81.67983934894232 0.0 0.1480676293373108
81.81275093173015 0.0 0.17464994589487712
81.93950120225577 0.0 0.2
81.93950120225577 0.0 0.2
81.93950120225577 0.0 0.2
81.93950120225577 0.0 0.2
81.93950120225577 0.0 0.2
81.98000392928748 1.0 0.00810054540634153
82.09840066845247 1.0 0.03177989323933916
82.22937259689002 1.0 0.05797427892684931
82.36211945230791 1.0 0.08452365001042678
start a slew
moving from elevation 82.6 deg to 84.9 deg
This will run for 41 seconds
82.70728518782227 0.0 0.016695070266723632
83.84029253540604 1.0 0.04329653978347775
84.03536755063304 1.0 0.0823115428288777
84.17531170664398 1.0 0.11030037403106685
84.30900595802235 1.0 0.1370392243067423
84.44200603383946 1.0 0.16363923947016396
84.

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=2070072708, ack=<SalRetCode.CMD_FAILED: -302>, error=6612, result='Rejected : command not allowed in current state')

In [120]:
print(Time.now())

2021-05-26 04:36:05.671703


#### Plot the above process

In [ ]:
end = Time(datetime.now(), scale='tai')
start = end - timedelta(seconds=1000)

dfm = await client.select_time_series('lsst.sal.MTMount.elevation', '*', start, end, csc_index)
dfm1m3 = await client.select_time_series('lsst.sal.MTM1M3.logevent_appliedElevationForces', '*', start, end, csc_index)
dfm2 = await client.select_time_series('lsst.sal.MTM2.axialForce', '*', start, end, csc_index)
dfh = await client.select_time_series('lsst.sal.MTHexapod.application', '*', start, end, csc_index)

idx1=dfh.MTHexapodID==1
dfh1 = dfh[idx1]
idx2=dfh.MTHexapodID==2
dfh2 = dfh[idx2]

fig, ax = plt.subplots(1,1, figsize=(15,4))
plt.plot(dfm.actualPosition, '--', label='mount elevation')
plt.plot(dfm1m3.yForces0, label='M1M3 elevation y-force 101')
plt.plot(dfm2.lutGravity0, label='M2 elevation force B1')
plt.plot(dfh1.position1, label='Camera hexapod y')
plt.plot(dfh2.position1, label='M2 hexapod y')
plt.grid()
plt.legend()

#### Check if the telescope is in tracking mode. If yes, need to stop stacking.

The alternative is to check "MT Mount status" dash board on Chronograf. Make sure there are three "False".

In [ ]:
mountStatus = await mount.evt_axesInPosition.aget(timeout=5.)
rotStatus = await rot.evt_inPosition.aget(timeout=5.)
trackingStatus = mountStatus.elevation and mountStatus.azimuth and rotStatus.inPosition
print('Are we tracking?', trackingStatus)

In [115]:
await ptg.cmd_stopTracking.set_start(timeout=5.)

#### Close up. Put all simulators to standby

In [ ]:
await salobj.set_summary_state(ptg, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(mount, salobj.State.STANDBY)